In [5]:
import pandas as pd
import re

def preprocess_csv(file_path):
    df = pd.read_csv(file_path)

    # Strip and rename columns
    df.columns = df.columns.str.strip()

    # Drop duplicates
    df.drop_duplicates(inplace=True)

    # Strip whitespace from string fields
    str_cols = df.select_dtypes(include='object').columns
    df[str_cols] = df[str_cols].apply(lambda x: x.str.strip())

    # Clean lyrics: lowercase, remove special characters except basic punctuation
    if 'Lyrics' in df.columns:
        df['Lyrics'] = df['Lyrics'].astype(str).str.lower()
        df['Lyrics'] = df['Lyrics'].apply(lambda text: re.sub(r'[^a-zA-Z0-9\s\']', '', text))

    # Convert numeric columns
    for col in ['key', 'energy', 'tempo']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Drop rows with missing essential fields
    essential = ['song', 'artist', 'key', 'energy', 'tempo', 'Lyrics']
    df.dropna(subset=essential, inplace=True)

    return df


In [6]:
import pandas as pd
import numpy as np
print("🔍 Column Headers Identification Tool")
print("="*50)

# Load the dataset (use cleaned version if available, otherwise original)
try:
    df = pd.read_csv('cleaned_data.csv')
    print("✅ Loaded cleaned dataset: 'cleaned_data.csv'")
    data_source = "cleaned_data.csv"
except FileNotFoundError:
    try:
        df = pd.read_csv('test data.csv', encoding='latin1')
        print("✅ Loaded original dataset: 'test data.csv'")
        data_source = "test data.csv"
    except FileNotFoundError:
        print("❌ No dataset found! Please ensure 'cleaned_data.csv' or 'test data.csv' exists.")
        exit()

print(f"📊 Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print("="*70)

# 1. BASIC COLUMN INFORMATION
print("\n🏷️  COLUMN HEADERS OVERVIEW")
print("="*40)
print(f"Total Columns: {len(df.columns)}")
print(f"Data Source: {data_source}")

# 2. LIST ALL COLUMN NAMES
print("\n📋 ALL COLUMN NAMES:")
print("-" * 30)
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

# 3. COLUMN NAMES WITH DATA TYPES
print(f"\n🔤 COLUMN NAMES WITH DATA TYPES:")
print("-" * 45)
for i, (col, dtype) in enumerate(zip(df.columns, df.dtypes), 1):
    print(f"{i:2d}. {col:<25} | {str(dtype):<15}")

# 4. CATEGORIZE COLUMNS BY DATA TYPE
print(f"\n📊 COLUMNS CATEGORIZED BY DATA TYPE:")
print("-" * 45)

# Numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\n🔢 NUMERIC COLUMNS ({len(numeric_cols)}):")
for i, col in enumerate(numeric_cols, 1):
    print(f"   {i}. {col}")

# Text/Object columns
text_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\n📝 TEXT/CATEGORICAL COLUMNS ({len(text_cols)}):")
for i, col in enumerate(text_cols, 1):
    print(f"   {i}. {col}")

# DateTime columns (if any)
datetime_cols = df.select_dtypes(include=['datetime64']).columns.tolist()
if datetime_cols:
    print(f"\n📅 DATETIME COLUMNS ({len(datetime_cols)}):")
    for i, col in enumerate(datetime_cols, 1):
        print(f"   {i}. {col}")

# Boolean columns (if any)
bool_cols = df.select_dtypes(include=['bool']).columns.tolist()
if bool_cols:
    print(f"\n☑️  BOOLEAN COLUMNS ({len(bool_cols)}):")
    for i, col in enumerate(bool_cols, 1):
        print(f"   {i}. {col}")

# 5. DETAILED COLUMN ANALYSIS
print(f"\n🔍 DETAILED COLUMN ANALYSIS:")
print("=" * 60)
print(f"{'Column Name':<25} | {'Type':<12} | {'Non-Null':<8} | {'Unique':<8}")
print("-" * 60)

for col in df.columns:
    col_type = str(df[col].dtype)
    non_null_count = df[col].count()
    unique_count = df[col].nunique()
    print(f"{col:<25} | {col_type:<12} | {non_null_count:<8} | {unique_count:<8}")

# 6. COLUMN NAME PATTERNS
print(f"\n🔍 COLUMN NAME PATTERNS:")
print("-" * 35)

# Check for common naming patterns
has_spaces = [col for col in df.columns if ' ' in col]
has_underscores = [col for col in df.columns if '_' in col]
has_numbers = [col for col in df.columns if any(char.isdigit() for char in col)]
all_uppercase = [col for col in df.columns if col.isupper()]
all_lowercase = [col for col in df.columns if col.islower()]

print(f"Columns with spaces: {len(has_spaces)}")
if has_spaces:
    print(f"   Examples: {has_spaces[:3]}")

print(f"Columns with underscores: {len(has_underscores)}")
if has_underscores:
    print(f"   Examples: {has_underscores[:3]}")

print(f"Columns with numbers: {len(has_numbers)}")
if has_numbers:
    print(f"   Examples: {has_numbers[:3]}")

print(f"All uppercase columns: {len(all_uppercase)}")
print(f"All lowercase columns: {len(all_lowercase)}")

# 7. SAMPLE DATA FOR EACH COLUMN
print(f"\n👀 SAMPLE DATA FOR EACH COLUMN:")
print("=" * 50)

for col in df.columns:
    print(f"\n🏷️  Column: '{col}'")
    print(f"   Data Type: {df[col].dtype}")
    print(f"   Sample Values:")
    
    # Show first 3 non-null unique values
    sample_values = df[col].dropna().unique()[:3]
    for i, val in enumerate(sample_values, 1):
        # Truncate long values
        val_str = str(val)[:50] + "..." if len(str(val)) > 50 else str(val)
        print(f"      {i}. {val_str}")
    
    if len(sample_values) == 0:
        print("      (All values are null)")

# 8. EXPORT COLUMN INFORMATION
print(f"\n💾 EXPORTING COLUMN INFORMATION:")
print("-" * 40)

# Create column summary DataFrame
column_summary = pd.DataFrame({
    'Column_Name': df.columns,
    'Data_Type': df.dtypes.astype(str),
    'Non_Null_Count': [df[col].count() for col in df.columns],
    'Null_Count': [df[col].isnull().sum() for col in df.columns],
    'Unique_Values': [df[col].nunique() for col in df.columns],
    'Sample_Value': [str(df[col].dropna().iloc[0]) if df[col].count() > 0 else 'No data' for col in df.columns]
})

# Save to CSV
column_summary.to_csv('column_information.csv', index=False)
print("✅ Column information saved to 'column_information.csv'")

# Save column names only
with open('column_names.txt', 'w') as f:
    f.write("Column Names:\n")
    f.write("=" * 20 + "\n")
    for i, col in enumerate(df.columns, 1):
        f.write(f"{i}. {col}\n")

print("✅ Column names saved to 'column_names.txt'")

# 9. FINAL SUMMARY
print(f"\n" + "="*60)
print("🎯 COLUMN IDENTIFICATION COMPLETE!")
print("="*60)
print(f"✅ Total Columns Identified: {len(df.columns)}")
print(f"✅ Numeric Columns: {len(numeric_cols)}")
print(f"✅ Text/Categorical Columns: {len(text_cols)}")
print(f"✅ Column Information Exported: ✅")
print(f"✅ Ready for Data Analysis: ✅")
print("="*60)


🔍 Column Headers Identification Tool
✅ Loaded cleaned dataset: 'cleaned_data.csv'
📊 Dataset Shape: 3 rows × 19 columns

🏷️  COLUMN HEADERS OVERVIEW
Total Columns: 19
Data Source: cleaned_data.csv

📋 ALL COLUMN NAMES:
------------------------------
 1. artist
 2. song
 3. duration_ms
 4. explicit
 5. year
 6. popularity
 7. danceability
 8. energy
 9. key
10. loudness
11. mode
12. speechiness
13. acousticness
14. instrumentalness
15. liveness
16. valence
17. tempo
18. genre
19. lyrics

🔤 COLUMN NAMES WITH DATA TYPES:
---------------------------------------------
 1. artist                    | object         
 2. song                      | object         
 3. duration_ms               | float64        
 4. explicit                  | bool           
 5. year                      | float64        
 6. popularity                | float64        
 7. danceability              | float64        
 8. energy                    | float64        
 9. key                       | float64        


In [7]:
import pandas as pd
import numpy as np

print("🎵 FILTERING SONGS WITH YEAR < 2014")
print("="*50)

# Set pandas display options to show all data clearly
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', 50)

# Load your dataset - try multiple sources
df = None
data_source = ""

try:
    # Try corrected file first
    df = pd.read_csv('music_data_corrected.csv')
    data_source = "music_data_corrected.csv"
    print(f"✅ Loaded: {data_source}")
except FileNotFoundError:
    try:
        # Try cleaned data
        df = pd.read_csv('cleaned_data.csv')
        data_source = "cleaned_data.csv"
        print(f"✅ Loaded: {data_source}")
    except FileNotFoundError:
        try:
            # Try verified dataset
            df = pd.read_csv('dataset_all_columns_verified.csv')
            data_source = "dataset_all_columns_verified.csv"
            print(f"✅ Loaded: {data_source}")
        except FileNotFoundError:
            try:
                # Try as Excel file
                df = pd.read_excel('test data.csv', engine='openpyxl')
                data_source = "test data.csv (Excel format)"
                print(f"✅ Loaded: {data_source}")
            except:
                try:
                    # Last resort - original CSV
                    df = pd.read_csv('test data.csv', encoding='latin1')
                    data_source = "test data.csv"
                    print(f"✅ Loaded: {data_source}")
                except Exception as e:
                    print(f"❌ Could not load any dataset: {e}")
                    print("Creating sample data for demonstration...")
                    
                    # Create sample data
                    sample_data = {
                        'artist': ['The Beatles', 'Elvis Presley', 'Michael Jackson', 'Madonna', 'Taylor Swift', 'Ed Sheeran'],
                        'song': ['Hey Jude', 'Can\'t Help Falling in Love', 'Billie Jean', 'Like a Virgin', 'Shake It Off', 'Shape of You'],
                        'year': [1968, 1961, 1983, 1984, 2014, 2017],
                        'popularity': [85, 90, 88, 82, 95, 92],
                        'duration_ms': [431000, 182000, 294000, 228000, 219000, 233000]
                    }
                    df = pd.DataFrame(sample_data)
                    data_source = "Sample dataset (created for demo)"

print(f"📊 Dataset Info: {df.shape[0]} rows × {df.shape[1]} columns")

# Check if 'year' and 'song' columns exist
year_col = None
song_col = None

# Find year column (case insensitive)
for col in df.columns:
    if str(col).lower().strip() in ['year', 'release_year', 'yr']:
        year_col = col
        break

# Find song column (case insensitive)  
for col in df.columns:
    if str(col).lower().strip() in ['song', 'track', 'title', 'song_name', 'track_name']:
        song_col = col
        break

if year_col is None:
    print("❌ ERROR: 'year' column not found!")
    print("Available columns:", list(df.columns))
    exit()

if song_col is None:
    print("❌ ERROR: 'song' column not found!")
    print("Available columns:", list(df.columns))
    exit()

print(f"✅ Found year column: '{year_col}'")
print(f"✅ Found song column: '{song_col}'")

# Check data types and clean year column
print(f"\n🔍 YEAR COLUMN ANALYSIS:")
print(f"   Data type: {df[year_col].dtype}")
print(f"   Sample values: {df[year_col].dropna().unique()[:10]}")

# Convert year to numeric if needed
df[year_col] = pd.to_numeric(df[year_col], errors='coerce')

# Check for missing values in year column
missing_years = df[year_col].isnull().sum()
if missing_years > 0:
    print(f"⚠️ Warning: {missing_years} songs have missing year values")

# Filter songs with year < 2014
print(f"\n🎯 FILTERING SONGS WITH YEAR < 2014:")
print("-" * 40)

# Create the filter
songs_before_2014 = df[df[year_col] < 2014].copy()

print(f"📈 FILTER RESULTS:")
print(f"   Total songs in dataset: {len(df)}")
print(f"   Songs with year < 2014: {len(songs_before_2014)}")
print(f"   Percentage: {len(songs_before_2014)/len(df)*100:.1f}%")

if len(songs_before_2014) == 0:
    print("❌ No songs found with year < 2014")
    print("\n📊 Year distribution in your dataset:")
    year_stats = df[year_col].describe()
    print(year_stats)
else:
    # Sort by year for better display
    songs_before_2014 = songs_before_2014.sort_values(year_col)
    
    print(f"\n🎵 ALL SONGS WITH YEAR < 2014:")
    print("=" * 60)
    
    # Display methods
    
    # Method 1: Simple list of songs with year
    print("📋 SONG LIST (Song - Artist - Year):")
    print("-" * 45)
    for idx, row in songs_before_2014.iterrows():
        artist = row.get('artist', 'Unknown Artist') if 'artist' in df.columns else 'Unknown Artist'
        song = row[song_col]
        year = int(row[year_col]) if pd.notna(row[year_col]) else 'Unknown'
        print(f"   • {song} - {artist} ({year})")
    
    # Method 2: Detailed table view
    print(f"\n📊 DETAILED TABLE VIEW:")
    print("-" * 50)
    
    # Select relevant columns for display
    display_cols = [song_col, year_col]
    if 'artist' in df.columns:
        display_cols.insert(1, 'artist')
    if 'popularity' in df.columns:
        display_cols.append('popularity')
    if 'genre' in df.columns:
        display_cols.append('genre')
    
    # Only include columns that actually exist
    display_cols = [col for col in display_cols if col in songs_before_2014.columns]
    
    print(songs_before_2014[display_cols].to_string(index=False))
    
    # Method 3: Year-wise breakdown
    print(f"\n📅 YEAR-WISE BREAKDOWN:")
    print("-" * 25)
    year_counts = songs_before_2014[year_col].value_counts().sort_index()
    for year, count in year_counts.items():
        year_int = int(year) if pd.notna(year) else 'Unknown'
        print(f"   {year_int}: {count} songs")
    
    # Method 4: Just song names (clean list)
    print(f"\n🎼 SONG NAMES ONLY:")
    print("-" * 20)
    song_list = songs_before_2014[song_col].tolist()
    for i, song in enumerate(song_list, 1):
        print(f"{i:3d}. {song}")

# Save filtered results
print(f"\n💾 SAVING FILTERED RESULTS:")
print("-" * 30)

if len(songs_before_2014) > 0:
    # Save to CSV
    output_file = 'songs_before_2014.csv'
    songs_before_2014.to_csv(output_file, index=False)
    print(f"✅ Filtered songs saved to: {output_file}")
    
    # Save just song names to text file
    song_names_file = 'song_names_before_2014.txt'
    with open(song_names_file, 'w', encoding='utf-8') as f:
        f.write("Songs with Year < 2014\n")
        f.write("=" * 25 + "\n\n")
        for i, song in enumerate(song_list, 1):
            f.write(f"{i}. {song}\n")
    print(f"✅ Song names saved to: {song_names_file}")

print(f"\n🎉 FILTERING COMPLETE!")
print("=" * 25)
print(f"✅ Found {len(songs_before_2014)} songs with year < 2014")
print("✅ Results displayed and saved")
print("✅ Ready for further analysis!")

# Summary statistics
if len(songs_before_2014) > 0:
    print(f"\n📊 SUMMARY STATISTICS:")
    print("-" * 25)
    oldest_year = songs_before_2014[year_col].min()
    newest_year = songs_before_2014[year_col].max()
    print(f"   Oldest song year: {int(oldest_year) if pd.notna(oldest_year) else 'Unknown'}")
    print(f"   Newest song year: {int(newest_year) if pd.notna(newest_year) else 'Unknown'}")
    print(f"   Year range: {int(newest_year - oldest_year) if pd.notna(oldest_year) and pd.notna(newest_year) else 'Unknown'} years")
    
    if 'popularity' in songs_before_2014.columns:
        avg_popularity = songs_before_2014['popularity'].mean()
        print(f"   Average popularity: {avg_popularity:.1f}")

🎵 FILTERING SONGS WITH YEAR < 2014
✅ Loaded: cleaned_data.csv
📊 Dataset Info: 3 rows × 19 columns
✅ Found year column: 'year'
✅ Found song column: 'song'

🔍 YEAR COLUMN ANALYSIS:
   Data type: float64
   Sample values: [2014.  2019.  2016.5]

🎯 FILTERING SONGS WITH YEAR < 2014:
----------------------------------------
📈 FILTER RESULTS:
   Total songs in dataset: 3
   Songs with year < 2014: 0
   Percentage: 0.0%
❌ No songs found with year < 2014

📊 Year distribution in your dataset:
count       3.00
mean     2016.50
std         2.50
min      2014.00
25%      2015.25
50%      2016.50
75%      2017.75
max      2019.00
Name: year, dtype: float64

💾 SAVING FILTERED RESULTS:
------------------------------

🎉 FILTERING COMPLETE!
✅ Found 0 songs with year < 2014
✅ Results displayed and saved
✅ Ready for further analysis!


In [8]:
import pandas as pd

def print_songs_with_sorry(file_path):
    df = pd.read_csv(file_path)

    # Print column names to debug
    print("Factors that have been considered to filter songs:", df.columns.tolist())

    # Column definitions (update if needed)
    song_col = 'song'
    key_col = 'key'
    energy_col = 'energy'
    lyrics_col = 'Lyrics'
    tempo_col = 'tempo'

    # Drop missing data
    df.dropna(subset=[song_col, key_col, energy_col, lyrics_col,tempo_col], inplace=True)

    # Ensure numeric types
    df[key_col] = pd.to_numeric(df[key_col], errors='coerce')
    df[energy_col] = pd.to_numeric(df[energy_col], errors='coerce')
    df[tempo_col] = pd.to_numeric(df[energy_col], errors='coerce')

    # Filter
    sad_songs = df[
        (df[key_col] < 5) &
        (df[energy_col] < 0.4) &
        (df[tempo_col] < 90) &
        (df[lyrics_col].str.contains(r'\bsorry|cry|lonely|sad|goodbye|fall\b', case=False, na=False))
    ]
    happy_songs = df[
        (df[energy_col] > 0.6) &
        # (df[tempo_col] > 110) &
        (df[key_col].isin([2, 4, 7, 11])) &
        (df[lyrics_col].str.contains(r'\b(fun|smile|party|sunshine|joy)\b', case=False, na=False))
    ]
    energetic_songs = df[
        (df[energy_col] > 0.7) &
        (df[tempo_col] < 120) &
        (df[key_col].isin([2, 4, 7, 11])) &
        (df[lyrics_col].str.contains(r'\b(fire|dance|wild|burn|crazy|alive)\b', case=False, na=False))
    ]
    romantic_songs = df[
        (df[energy_col].between(0.4, 0.7)) &
        (df[tempo_col] < 80) &
        (df[key_col].isin([0, 7, 9])) &
        (df[lyrics_col].str.contains(r'\b(love|heart|kiss|baby|darling|hold)\b', case=False, na=False))
    ]
    relaxing_songs = df[
        (df[energy_col] < 0.5) &
        (df[tempo_col] < 70) &
        (df[key_col].isin([5, 7])) &
        (df[lyrics_col].str.contains(r'\b(calm|peace|slow|breeze|chill|dream)\b', case=False, na=False))
    ]

    categories_to_print = {
        "Sad": (sad_songs, "☹️"),
        "Energetic": (energetic_songs, "⚡"),
        "Happy": (happy_songs, "😊"),
        "Romantic": (romantic_songs, "❤️"),
        "Relaxing": (relaxing_songs, "😌")
    }

    # Print results efficiently
    for name, (songs_df, emoji) in categories_to_print.items():
        if songs_df.empty:
            print(f"\nNo songs found matching the conditions for '{name}' category.")
        else:
            print(f"\n{emoji} Category of {name} songs:")
            for song in songs_df[song_col]:
                print(song)

# Example usage
print_songs_with_sorry("test data.csv")
# print_songs_with_sorry("song_dataset.csv")

Factors that have been considered to filter songs: ['singer', 'song', 'energy', 'key', 'tempo', 'Lyrics']

☹️ Category of Sad songs:
Someone Like You
Skyfall
Too Good At Goodbyes
See You Again (feat. Charlie Puth)
In the End
We Don't Talk Anymore (feat. Selena Gomez)
Show Me the Meaning of Being Lonely
Summertime Sadness (Cedric Gervais Remix)
When I Was Your Man
Talking to the Moon
Back To December
I Knew You Were Trouble.
Rolling in the Deep
Set Fire to the Rain
Hello
Stan
Mockingbird
Love The Way You Lie
Mercy
Love Yourself
Call Out My Name
Same Old Love
Chandelier
The Lazy Song
Me And My Broken Heart

⚡ Category of Energetic songs:
Numb
Roar
Hips Don't Lie (feat. Wyclef Jean)
Live While We're Young
On The Floor
Bad Blood
Look What You Made Me Do
Rumour Has It
IDGAF
Without Me
Lose Yourself - From "8 Mile" Soundtrack
The Monster
Uptown Funk (feat. Bruno Mars)
Dark Horse
Animals
Starboy
Tonight (I'm Fuckin' You)
Bailando
Cheap Thrills (feat. Sean Paul)
Dusk Till Dawn (Radio Edit)
Bad

C:\Users\DINESHI\AppData\Local\Temp\ipykernel_17412\4075365895.py:35: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (df[lyrics_col].str.contains(r'\b(fun|smile|party|sunshine|joy)\b', case=False, na=False))
C:\Users\DINESHI\AppData\Local\Temp\ipykernel_17412\4075365895.py:41: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (df[lyrics_col].str.contains(r'\b(fire|dance|wild|burn|crazy|alive)\b', case=False, na=False))
C:\Users\DINESHI\AppData\Local\Temp\ipykernel_17412\4075365895.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (df[lyrics_col].str.contains(r'\b(love|heart|kiss|baby|darling|hold)\b', case=False, na=False))
C:\Users\DINESHI\AppData\Local\Temp\ipykernel_17412\4075365895.py:53: UserWarning: This pattern is inte

In [9]:
import pandas as pd
import re

def calculate_mood_weights(file_path):
    df = pd.read_csv(file_path)

    # Define columns
    song_col = 'song'
    key_col = 'key'
    energy_col = 'energy'
    lyrics_col = 'Lyrics'
    tempo_col = 'tempo'

    # Clean and convert types
    df.dropna(subset=[song_col, key_col, energy_col, lyrics_col, tempo_col], inplace=True)
    df[key_col] = pd.to_numeric(df[key_col], errors='coerce')
    df[energy_col] = pd.to_numeric(df[energy_col], errors='coerce')
    df[tempo_col] = pd.to_numeric(df[tempo_col], errors='coerce')

    # Helper function to check condition
    def contains_keywords(text, keywords):
        return bool(re.search(rf'\b({keywords})\b', str(text), re.IGNORECASE))

    # Initialize mood weights
    mood_weights = {
        'sad_weight': [],
        'happy_weight': [],
        'energetic_weight': [],
        'romantic_weight': [],
        'relaxing_weight': []
    }

    # Loop through each song
    for _, row in df.iterrows():
        lyrics = row[lyrics_col]
        key = row[key_col]
        energy = row[energy_col]
        tempo = row[tempo_col]

        # Sad mood
        sad_count = sum([
            key < 5,
            energy < 0.4,
            tempo < 90,
            contains_keywords(lyrics, "sorry|cry|lonely|sad|goodbye|fall")
        ])
        mood_weights['sad_weight'].append(sad_count / 4)

        # Happy mood
        happy_count = sum([
            energy > 0.6,
            tempo > 110,
            key in [2, 4, 7, 11],
            contains_keywords(lyrics, "fun|smile|party|sunshine|joy")
        ])
        mood_weights['happy_weight'].append(happy_count / 4)

        # Energetic mood
        energetic_count = sum([
            energy > 0.7,
            tempo < 120,
            key in [2, 4, 7, 11],
            contains_keywords(lyrics, "fire|dance|wild|burn|crazy|alive")
        ])
        mood_weights['energetic_weight'].append(energetic_count / 4)

        # Romantic mood
        romantic_count = sum([
            0.4 <= energy <= 0.7,
            80 <= tempo <= 115,
            key in [0, 7, 9],
            contains_keywords(lyrics, "love|heart|kiss|baby|darling|hold")
        ])
        mood_weights['romantic_weight'].append(romantic_count / 4)

        # Relaxing mood
        relaxing_count = sum([
            energy < 0.5,
            70 <= tempo <= 110,
            key in [5, 7],
            contains_keywords(lyrics, "calm|peace|slow|breeze|chill|dream")
        ])
        mood_weights['relaxing_weight'].append(relaxing_count / 4)

    # Add mood weights to DataFrame
    for mood, values in mood_weights.items():
        df[mood] = values

    # Save or display table
    # print("\nMood Weights Table:")
    # print(df[[song_col, 'sad_weight', 'happy_weight', 'energetic_weight', 'romantic_weight', 'relaxing_weight']])

    return df

# Example usage
calculate_mood_weights("test data.csv")


,singer,song,energy,key,tempo,Lyrics,sad_weight,happy_weight,energetic_weight,romantic_weight,relaxing_weight
0,Taylor Swift,Shake It Off,0.85,4,125,I stay out too lateGot nothing in my brainThat...,0.25,1.00,0.50,0.25,0.00
1,Taylor Swift,Cruel Summer,0.80,9,125,Fever dream high in the quiet of the nightYou ...,0.00,0.75,0.25,0.50,0.25
2,Adele,Someone Like You,0.30,2,70,I heard that youre settled downThat you found ...,1.00,0.25,0.50,0.25,0.50
3,Adele,Skyfall,0.30,3,75,This is the endHold your breath and count to t...,1.00,0.00,0.25,0.25,0.50
4,Sam Smith,Too Good At Goodbyes,0.30,2,80,You must think that Im stupidYou must think th...,1.00,0.25,0.50,0.50,0.50
5,Wiz Khalifa,See You Again (feat. Charlie Puth),0.30,2,80,Its been a long day without you my friendAnd I...,0.75,0.25,0.50,0.50,0.50
6,Shawn Mendes,Treat You Better,0.50,9,100,I wont lie to youI know hes just not right for...,0.00,0.00,0.25,1.00,0.25
7,Maroon 5,Girls Like You (feat. Cardi B) - Cardi B Version,0.60,9,90,Spent 24 hours I need more hours with youYou s...,0.00,0.25,0.25,1.00,0.25
8,Linkin Park,Numb,0.90,4,130,Im tired of being what you want me to beFeelin...,0.25,0.75,0.75,0.00,0.00
9,Linkin Park,In the End,0.36,2,82,One thing I dont know whyIt doesnt even matter...,1.00,0.25,0.50,0.50,0.75


In [10]:
import pandas as pd
import re

def calculate_mood_weights(file_path):
    df = pd.read_csv(file_path)

    # Define columns
    song_col = 'song'
    key_col = 'key'
    energy_col = 'energy'
    lyrics_col = 'Lyrics'
    tempo_col = 'tempo'

    # Clean and convert types
    df.dropna(subset=[song_col, key_col, energy_col, lyrics_col, tempo_col], inplace=True)
    df[key_col] = pd.to_numeric(df[key_col], errors='coerce')
    df[energy_col] = pd.to_numeric(df[energy_col], errors='coerce')
    df[tempo_col] = pd.to_numeric(df[tempo_col], errors='coerce')

    # Helper function to check condition
    def contains_keywords(text, keywords):
        return bool(re.search(rf'\b({keywords})\b', str(text), re.IGNORECASE))

    # Initialize mood weights
    moods = ['sad', 'happy', 'energetic', 'romantic', 'relaxing']
    mood_weights = {f'{m}_weight': [] for m in moods}

    # Loop through each song
    for _, row in df.iterrows():
        lyrics = row[lyrics_col]
        key = row[key_col]
        energy = row[energy_col]
        tempo = row[tempo_col]

        # Sad mood
        sad_count = sum([
            key < 5,
            energy < 0.4,
            tempo < 90,
            contains_keywords(lyrics, "sorry|cry|lonely|sad|goodbye|fall")
        ])
        mood_weights['sad_weight'].append(sad_count / 4)

        # Happy mood
        happy_count = sum([
            energy > 0.6,
            tempo > 110,
            key in [2, 4, 7, 11],
            contains_keywords(lyrics, "fun|smile|party|sunshine|joy")
        ])
        mood_weights['happy_weight'].append(happy_count / 4)

        # Energetic mood
        energetic_count = sum([
            energy > 0.7,
            tempo < 120,
            key in [2, 4, 7, 11],
            contains_keywords(lyrics, "fire|dance|wild|burn|crazy|alive")
        ])
        mood_weights['energetic_weight'].append(energetic_count / 4)

        # Romantic mood
        romantic_count = sum([
            0.4 <= energy <= 0.7,
            80 <= tempo <= 115,
            key in [0, 7, 9],
            contains_keywords(lyrics, "love|heart|kiss|baby|darling|hold")
        ])
        mood_weights['romantic_weight'].append(romantic_count / 4)

        # Relaxing mood
        relaxing_count = sum([
            energy < 0.5,
            70 <= tempo <= 110,
            key in [5, 7],
            contains_keywords(lyrics, "calm|peace|slow|breeze|chill|dream")
        ])
        mood_weights['relaxing_weight'].append(relaxing_count / 4)

    # Add mood weights to DataFrame
    for mood, values in mood_weights.items():
        df[mood] = values

    # Determine major and second major feelings
    def get_top_two_moods(row):
        weights = {m: row[f'{m}_weight'] for m in moods}
        sorted_moods = sorted(weights.items(), key=lambda x: x[1], reverse=True)
        return pd.Series([sorted_moods[0][0].replace("_weight", ""), sorted_moods[1][0].replace("_weight", "")])

    df[['major_feeling', 'second_major_feeling']] = df.apply(get_top_two_moods, axis=1)

    # Final table output
    final_columns = [song_col] + [f'{m}_weight' for m in moods] + ['major_feeling', 'second_major_feeling']
    print("\n🎵 Final Mood Analysis Table:")
    # print(df[final_columns])

    return df[final_columns]

# Example usage:
calculate_mood_weights("test data.csv")



🎵 Final Mood Analysis Table:


,song,sad_weight,happy_weight,energetic_weight,romantic_weight,relaxing_weight,major_feeling,second_major_feeling
0,Shake It Off,0.25,1.00,0.50,0.25,0.00,happy,energetic
1,Cruel Summer,0.00,0.75,0.25,0.50,0.25,happy,romantic
2,Someone Like You,1.00,0.25,0.50,0.25,0.50,sad,energetic
3,Skyfall,1.00,0.00,0.25,0.25,0.50,sad,relaxing
4,Too Good At Goodbyes,1.00,0.25,0.50,0.50,0.50,sad,energetic
5,See You Again (feat. Charlie Puth),0.75,0.25,0.50,0.50,0.50,sad,energetic
6,Treat You Better,0.00,0.00,0.25,1.00,0.25,romantic,energetic
7,Girls Like You (feat. Cardi B) - Cardi B Version,0.00,0.25,0.25,1.00,0.25,romantic,happy
8,Numb,0.25,0.75,0.75,0.00,0.00,happy,energetic
9,In the End,1.00,0.25,0.50,0.50,0.75,sad,relaxing


In [11]:
import pandas as pd
import re

def generate_feeling_tables(file_path):
    df = pd.read_csv(file_path)

    # Clean data
    df = df[['song', 'key', 'energy', 'tempo', 'Lyrics']].dropna()
    df['key'] = pd.to_numeric(df['key'], errors='coerce')
    df['energy'] = pd.to_numeric(df['energy'], errors='coerce')
    df['tempo'] = pd.to_numeric(df['tempo'], errors='coerce')

    # Mood conditions
    moods = {
        'happy': [
            lambda r: r['energy'] > 0.6,
            lambda r: r['tempo'] > 110,
            lambda r: r['key'] in [2, 4, 7, 11],
            lambda r: bool(re.search(r'\b(fun|smile|party|sunshine|joy)\b', str(r['Lyrics']), re.IGNORECASE))
        ],
        'sad': [
            lambda r: r['key'] < 5,
            lambda r: r['energy'] < 0.4,
            lambda r: r['tempo'] < 90,
            lambda r: bool(re.search(r'\b(sorry|cry|lonely|sad|goodbye|fall)\b', str(r['Lyrics']), re.IGNORECASE))
        ],
        'romantic': [
            lambda r: 0.4 <= r['energy'] <= 0.7,
            lambda r: 80 <= r['tempo'] <= 115,
            lambda r: r['key'] in [0, 7, 9],
            lambda r: bool(re.search(r'\b(love|heart|kiss|baby|darling|hold)\b', str(r['Lyrics']), re.IGNORECASE))
        ],
        'relaxing': [
            lambda r: r['energy'] < 0.5,
            lambda r: 70 <= r['tempo'] <= 110,
            lambda r: r['key'] in [5, 7],
            lambda r: bool(re.search(r'\b(calm|peace|slow|breeze|chill|dream)\b', str(r['Lyrics']), re.IGNORECASE))
        ],
        'energetic': [
            lambda r: r['energy'] > 0.7,
            lambda r: r['tempo'] < 120,
            lambda r: r['key'] in [2, 4, 7, 11],
            lambda r: bool(re.search(r'\b(fire|dance|wild|burn|crazy|alive)\b', str(r['Lyrics']), re.IGNORECASE))
        ]
    }

    # Calculate mood weights
    for mood, checks in moods.items():
        df[f'{mood}_weight'] = df.apply(lambda r: sum(check(r) for check in checks) / len(checks), axis=1)

    # Determine major feeling and its weight
    weight_cols = [f"{m}_weight" for m in moods]
    df['major_feeling'] = df[weight_cols].idxmax(axis=1).str.replace('_weight', '')
    df['major_weight'] = df[weight_cols].max(axis=1)

    # Generate individual tables
    mood_tables = {}
    for mood in moods.keys():
        table = df[df['major_feeling'] == mood][['major_feeling', 'song', 'major_weight']]
        mood_tables[mood] = table.rename(columns={'major_weight': 'weight'}).sort_values(by='weight', ascending=False).reset_index(drop=True)

    return mood_tables

# Example usage:
tables = generate_feeling_tables("test data.csv")

# Print format like image (optional)
for mood, table in tables.items():
    print(f"\n🎵 {mood.capitalize()} Songs:")
    print(table.to_string(index=False))



🎵 Happy Songs:
major_feeling                                             song  weight
        happy                                     Shake It Off    1.00
        happy                         What Makes You Beautiful    1.00
        happy                           Live While We're Young    1.00
        happy               Hips Don't Lie (feat. Wyclef Jean)    1.00
        happy                                     On The Floor    1.00
        happy                                            IDGAF    1.00
        happy                                         Who Says    1.00
        happy                  There's Nothing Holdin' Me Back    1.00
        happy                   Uptown Funk (feat. Bruno Mars)    1.00
        happy                                      Bad Romance    1.00
        happy                                            Sugar    0.75
        happy                                        Bad Blood    0.75
        happy                               You Belong With M

In [12]:
from tabulate import tabulate

# To print tables nicely
for mood, table in tables.items():
    print(f"\n🎵 {mood.capitalize()} Songs:")
    print(tabulate(table, headers='keys', tablefmt='fancy_grid', showindex=False))



🎵 Happy Songs:
╒═════════════════╤══════════════════════════════════════════════════╤══════════╕
│ major_feeling   │ song                                             │   weight │
╞═════════════════╪══════════════════════════════════════════════════╪══════════╡
│ happy           │ Shake It Off                                     │     1    │
├─────────────────┼──────────────────────────────────────────────────┼──────────┤
│ happy           │ What Makes You Beautiful                         │     1    │
├─────────────────┼──────────────────────────────────────────────────┼──────────┤
│ happy           │ Live While We're Young                           │     1    │
├─────────────────┼──────────────────────────────────────────────────┼──────────┤
│ happy           │ Hips Don't Lie (feat. Wyclef Jean)               │     1    │
├─────────────────┼──────────────────────────────────────────────────┼──────────┤
│ happy           │ On The Floor                                     │     1    │


In [13]:
import streamlit as st
import pandas as pd
import re

def generate_feeling_tables(file_path):
    df = pd.read_csv(file_path)

    # Clean data
    df = df[['song', 'key', 'energy', 'tempo', 'Lyrics']].dropna()
    df['key'] = pd.to_numeric(df['key'], errors='coerce')
    df['energy'] = pd.to_numeric(df['energy'], errors='coerce')
    df['tempo'] = pd.to_numeric(df['tempo'], errors='coerce')

    # Mood conditions
    moods = {
        'happy': [
            lambda r: r['energy'] > 0.6,
            lambda r: r['tempo'] > 110,
            lambda r: r['key'] in [2, 4, 7, 11],
            lambda r: bool(re.search(r'\b(fun|smile|party|sunshine|joy)\b', str(r['Lyrics']), re.IGNORECASE))
        ],
        'sad': [
            lambda r: r['key'] < 5,
            lambda r: r['energy'] < 0.4,
            lambda r: r['tempo'] < 90,
            lambda r: bool(re.search(r'\b(sorry|cry|lonely|sad|goodbye|fall)\b', str(r['Lyrics']), re.IGNORECASE))
        ],
        'romantic': [
            lambda r: 0.4 <= r['energy'] <= 0.7,
            lambda r: 80 <= r['tempo'] <= 115,
            lambda r: r['key'] in [0, 7, 9],
            lambda r: bool(re.search(r'\b(love|heart|kiss|baby|darling|hold)\b', str(r['Lyrics']), re.IGNORECASE))
        ],
        'relaxing': [
            lambda r: r['energy'] < 0.5,
            lambda r: 70 <= r['tempo'] <= 110,
            lambda r: r['key'] in [5, 7],
            lambda r: bool(re.search(r'\b(calm|peace|slow|breeze|chill|dream)\b', str(r['Lyrics']), re.IGNORECASE))
        ],
        'energetic': [
            lambda r: r['energy'] > 0.7,
            lambda r: r['tempo'] < 120,
            lambda r: r['key'] in [2, 4, 7, 11],
            lambda r: bool(re.search(r'\b(fire|dance|wild|burn|crazy|alive)\b', str(r['Lyrics']), re.IGNORECASE))
        ]
    }

    # Calculate mood weights
    for mood, checks in moods.items():
        df[f'{mood}_weight'] = df.apply(lambda r: sum(check(r) for check in checks) / len(checks), axis=1)

    # Determine major feeling and its weight
    weight_cols = [f"{m}_weight" for m in moods]
    df['major_feeling'] = df[weight_cols].idxmax(axis=1).str.replace('_weight', '')
    df['major_weight'] = df[weight_cols].max(axis=1)

    # Generate individual tables
    mood_tables = {}
    for mood in moods.keys():
        table = df[df['major_feeling'] == mood][['major_feeling', 'song', 'major_weight']]
        mood_tables[mood] = table.rename(columns={'major_weight': 'weight'}).sort_values(by='weight', ascending=False).reset_index(drop=True)

    return mood_tables

# Streamlit UI
st.title("🎵 Mood-Based Song Tables")

uploaded_file = st.file_uploader("Upload your CSV file", type=["csv"])

if uploaded_file:
    mood_tables = generate_feeling_tables(uploaded_file)

    tabs = st.tabs(["😊 Happy", "😢 Sad", "💕 Romantic", "🧘 Relaxing", "⚡ Energetic"])
    mood_names = ["happy", "sad", "romantic", "relaxing", "energetic"]

    for i, mood in enumerate(mood_names):
        with tabs[i]:
            st.subheader(f"{mood.capitalize()} Songs")
            st.dataframe(mood_tables[mood])
else:
    st.info("Please upload a CSV file to see the mood-based song tables.")


2025-07-25 21:41:07.400 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 21:41:08.113 
  command:

    streamlit run C:\Users\DINESHI\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-25 21:41:08.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 21:41:08.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 21:41:08.115 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 21:41:08.115 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 21:41:08.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 21:41:08.118 Th

In [18]:
import pandas as pd
import re

def preprocess_csv(file_path):
    """
    Preprocess CSV file with music data including lyrics cleaning
    """
    df = pd.read_csv(file_path)

    # Strip and rename columns
    df.columns = df.columns.str.strip()

    # Drop duplicates
    df.drop_duplicates(inplace=True)

    # Strip whitespace from string fields
    str_cols = df.select_dtypes(include='object').columns
    df[str_cols] = df[str_cols].apply(lambda x: x.str.strip())

    # Clean lyrics: lowercase, remove special characters except basic punctuation
    if 'Lyrics' in df.columns:
        df['Lyrics'] = df['Lyrics'].astype(str).str.lower()
        df['Lyrics'] = df['Lyrics'].apply(lambda text: re.sub(r'[^a-zA-Z0-9\s\']', '', text))

    # Convert numeric columns
    for col in ['key', 'energy', 'tempo']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Drop rows with missing essential fields
    essential = ['singer', 'song', 'key', 'energy', 'tempo', 'Lyrics']
    df.dropna(subset=essential, inplace=True)

    return df

def main():
    # File path
    input_file = 'test data.csv'
    output_file = 'preprocessed_test_data.csv'
    
    try:
        print("Starting preprocessing...")
        
        # Load and check original data
        original_df = pd.read_csv(input_file)
        print(f"Original dataset shape: {original_df.shape}")
        print(f"Original columns: {list(original_df.columns)}")
        
        # Preprocess the data
        processed_df = preprocess_csv(input_file)
        
        # Display results
        print(f"\nProcessed dataset shape: {processed_df.shape}")
        print(f"Rows removed: {original_df.shape[0] - processed_df.shape[0]}")
        
        # Show data types after preprocessing
        print("\nData types after preprocessing:")
        print(processed_df.dtypes)
        
        # Show first few rows
        # print("\nFirst 5 rows of processed data:")
        # print(processed_df.head())
        
        # Save processed data
        processed_df.to_csv(output_file, index=False)
        print(f"\nProcessed data saved to: {output_file}")
        
        # Show summary statistics for numeric columns
        # numeric_cols = processed_df.select_dtypes(include=['float64', 'int64']).columns
        # if len(numeric_cols) > 0:
        #     print("\nSummary statistics for numeric columns:")
        #     print(processed_df[numeric_cols].describe())
            
    except FileNotFoundError:
        print(f"Error: File '{input_file}' not found. Please make sure the file exists in the current directory.")
    except Exception as e:
        print(f"Error during preprocessing: {str(e)}")

if __name__ == "__main__":
    main()

Starting preprocessing...
Original dataset shape: (100, 6)
Original columns: ['singer', 'song', 'energy', 'key', 'tempo', 'Lyrics']

Processed dataset shape: (100, 6)
Rows removed: 0

Data types after preprocessing:
singer     object
song       object
energy    float64
key         int64
tempo       int64
Lyrics     object
dtype: object

Processed data saved to: preprocessed_test_data.csv


In [20]:
import pandas as pd
import re

def preprocess_csv(file_path):
    """
    Preprocess CSV file with music data including lyrics cleaning
    """
    print("📊 Loading CSV file...")
    df = pd.read_csv(file_path)
    print("✅ CSV file loaded successfully")

    # Strip and rename columns
    print("🔧 Stripping whitespace from column names...")
    df.columns = df.columns.str.strip()
    print("✅ Column names cleaned")

    # Drop duplicates
    print("🗑️  Removing duplicate rows...")
    initial_rows = len(df)
    df.drop_duplicates(inplace=True)
    duplicates_removed = initial_rows - len(df)
    print(f"✅ Duplicates removed: {duplicates_removed} rows")

    # Strip whitespace from string fields
    print("✂️  Stripping whitespace from string fields...")
    str_cols = df.select_dtypes(include='object').columns
    df[str_cols] = df[str_cols].apply(lambda x: x.str.strip())
    print(f"✅ Whitespace stripped from {len(str_cols)} string columns")

    # Clean lyrics: lowercase, remove special characters except basic punctuation
    if 'Lyrics' in df.columns:
        print("🎵 Cleaning lyrics data...")
        df['Lyrics'] = df['Lyrics'].astype(str).str.lower()
        df['Lyrics'] = df['Lyrics'].apply(lambda text: re.sub(r'[^a-zA-Z0-9\s\']', '', text))
        print("✅ Lyrics cleaned (lowercase, special characters removed)")
    else:
        print("⚠️  'Lyrics' column not found - skipping lyrics cleaning")

    # Convert numeric columns
    print("🔢 Converting numeric columns...")
    numeric_conversions = 0
    for col in ['key', 'energy', 'tempo']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            numeric_conversions += 1
    print(f"✅ {numeric_conversions} columns converted to numeric")

    # Drop rows with missing essential fields
    print("🧹 Removing rows with missing essential fields...")
    rows_before = len(df)
    essential = ['singer', 'song', 'key', 'energy', 'tempo', 'Lyrics']
    df.dropna(subset=essential, inplace=True)
    rows_after = len(df)
    missing_removed = rows_before - rows_after
    print(f"✅ Rows with missing essential fields removed: {missing_removed} rows")

    return df

def main():
    # File path
    input_file = 'test data.csv'
    output_file = 'preprocessed_test_data.csv'
    
    try:
        print("Starting preprocessing...")
        
        # Load and check original data
        original_df = pd.read_csv(input_file)
        print(f"Original dataset shape: {original_df.shape}")
        print(f"Original columns: {list(original_df.columns)}")
        
        # Preprocess the data
        processed_df = preprocess_csv(input_file)
        
        # Display results
        print(f"\nProcessed dataset shape: {processed_df.shape}")
        print(f"Rows removed: {original_df.shape[0] - processed_df.shape[0]}")
        
        # Show data types after preprocessing
        print("\nData types after preprocessing:")
        print(processed_df.dtypes)
        
        # Show first few rows
        print("\nFirst 5 rows of processed data:")
        print(processed_df.head())
        
        # Save processed data
        processed_df.to_csv(output_file, index=False)
        print(f"\nProcessed data saved to: {output_file}")
        
        # Show summary statistics for numeric columns
        numeric_cols = processed_df.select_dtypes(include=['float64', 'int64']).columns
        if len(numeric_cols) > 0:
            print("\nSummary statistics for numeric columns:")
            print(processed_df[numeric_cols].describe())

        # Save processed data
        processed_df.to_csv(output_file, index=False)
        print(f"\nProcessed data saved to: {output_file}")
            
    except FileNotFoundError:
        print(f"Error: File '{input_file}' not found. Please make sure the file exists in the current directory.")
    except Exception as e:
        print(f"Error during preprocessing: {str(e)}")

if __name__ == "__main__":
    main()

Starting preprocessing...
Original dataset shape: (100, 6)
Original columns: ['singer', 'song', 'energy', 'key', 'tempo', 'Lyrics']
📊 Loading CSV file...
✅ CSV file loaded successfully
🔧 Stripping whitespace from column names...
✅ Column names cleaned
🗑️  Removing duplicate rows...
✅ Duplicates removed: 0 rows
✂️  Stripping whitespace from string fields...
✅ Whitespace stripped from 3 string columns
🎵 Cleaning lyrics data...
✅ Lyrics cleaned (lowercase, special characters removed)
🔢 Converting numeric columns...
✅ 3 columns converted to numeric
🧹 Removing rows with missing essential fields...
✅ Rows with missing essential fields removed: 0 rows

Processed dataset shape: (100, 6)
Rows removed: 0

Data types after preprocessing:
singer     object
song       object
energy    float64
key         int64
tempo       int64
Lyrics     object
dtype: object

First 5 rows of processed data:
         singer                  song  energy  key  tempo  \
0  Taylor Swift          Shake It Off    0.85  